# Lanternfish

## Part 1

In [1]:
test_data = "3,4,3,1,2"

with open("06_input.txt", "r") as f:
    full_data = f.read()
    full_data = full_data.splitlines()[0]

In [2]:
def simulate_population(initial_data, max_age=80):
    population = [int(i) for i in initial_data.split(",")]
    
    for day in range(max_age):
        for i in range(len(population)):
            age = population[i]
            if age == 0:
                age = 6
                population.append(8)
            else:
                age -= 1
                
            population[i] = age
            
    print(len(population))

In [3]:
simulate_population(test_data)

5934


In [4]:
simulate_population(full_data)

374994


## Part 2

In [5]:
# Using the same approach wont do...
simulate_population(test_data, max_age=160)

6311710


### Idea
Use a shifting register (from 0 to 8) of Generations.


`test_data` (`3,4,3,1,2`) would result in this starting register
```python
    generations = {
        0: 0,
        1: 1,
        2: 1,
        3: 2,
        4: 1,
        5: 0,
        6: 0,
        7: 0,
        8: 0
    }
```

or rather a stack like this:
```python
    generations = [0, 1, 1, 2, 1, 0, 0, 0, 0]
```

would look like this after two days:
```python
    #              0  1  2  3  4  5  6  7  8
    #                             +        +
    generations = [1, 2, 1, 0, 0, 1, 0, 0, 1]
```
Note the addition in two places: 6 (reset) and 8 (spawn)

In [10]:
class LanternFishPond():
    def __init__(self, data):
        #                 index:    0  1  2  3  4  5  6  7  8
        self.population_register = [0, 0, 0, 0, 0, 0, 0, 0, 0]
        self.age = 0
        
        starting_population = [int(fish_age) for fish_age in data.split(",")]
        for fish_age in starting_population:
            self.population_register[fish_age] += 1

    def step(self):
        # Effectively shifts the register by popping and appending.
        ready_to_spawn = self.population_register.pop(0)
        self.population_register[6] += ready_to_spawn
        self.population_register.append(ready_to_spawn)
        self.age += 1
    
    def run(self, days):
        for _ in range(days):
            self.step()
        # self.dump()
             
    def dump(self):
        total = sum(self.population_register)
        print(f"Day {self.age:03d} Population Report\n\t"
              f"Total of: {total} \n\t"
              f"Lanternfish Register: {self.population_register}")
    

In [7]:
pond = LanternFishPond(test_data)
pond.run(256)

Day 256 Population Report
	Total of: 26984457539 
	Lanternfish Register: [2376852196, 2731163883, 2897294544, 3164316379, 3541830408, 3681986557, 4275812629, 1985489551, 2329711392]


In [11]:
%%timeit
pond = LanternFishPond(full_data)
pond.run(256)

180 µs ± 2.22 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
